In [1]:
from network import Network
from layer_dropout import LayerDropout
from layer_wta import LayerWTA
from layer_lsh import LayerLSH
from layer import Layer

import numpy as np
from keras.utils import to_categorical
from keras.metrics import CategoricalAccuracy

import pickle

# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

hidden_units = 256
learning_rate = 0.01

function_num = 5
table_num = 6

dropout_probability = top_k = 1-(1-(1/2**function_num))**table_num

epochs = 25

In [2]:
neural_network = Network()
neural_network.addLayer(Layer(input_size, hidden_units))
neural_network.addLayer(Layer(hidden_units, hidden_units))
neural_network.addLayer(Layer(hidden_units, hidden_units))
neural_network.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy, epoch_time] = neural_network.fit(x_train, y_train, learning_rate=learning_rate, epochs = 1, progress=True)
y_hat = neural_network.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

Epoch  1: 100% |██████████| 60000/60000, Accuracy: 0.832;


0.8113


In [2]:
neural_network_dropout = Network()
neural_network_dropout.addLayer(LayerDropout(input_size, hidden_units, dropout_probability))
neural_network_dropout.addLayer(LayerDropout(hidden_units, hidden_units, dropout_probability))
neural_network_dropout.addLayer(LayerDropout(hidden_units, hidden_units, dropout_probability))
neural_network_dropout.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_dropout, epoch_time_dropout] = neural_network_dropout.fit(x_train, y_train, learning_rate=learning_rate, epochs = 1, progress=True)
y_hat = neural_network_dropout.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

Epoch  1: 100% |██████████| 60000/60000, Accuracy: 0.794;


0.8329


In [2]:
neural_network_wta = Network()
neural_network_wta.addLayer(LayerWTA(input_size, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_wta, epoch_time_wta] = neural_network_wta.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs, progress=False)
y_hat = neural_network_wta.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

epoch 1 has completed
epoch 2 has completed
epoch 3 has completed
epoch 4 has completed
epoch 5 has completed
epoch 6 has completed
epoch 7 has completed
epoch 8 has completed
epoch 9 has completed
epoch 10 has completed
epoch 11 has completed
epoch 12 has completed
epoch 13 has completed
epoch 14 has completed
epoch 15 has completed
epoch 16 has completed
epoch 17 has completed
epoch 18 has completed
epoch 19 has completed
epoch 20 has completed
epoch 21 has completed
epoch 22 has completed
epoch 23 has completed
epoch 24 has completed
epoch 25 has completed
0.9802


In [2]:
neural_network_lsh = Network()
neural_network_lsh.addLayer(LayerLSH(input_size, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_lsh, epoch_time_lsh] = neural_network_lsh.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs, progress=False)
y_hat = neural_network_lsh.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

epoch 1 has completed
epoch 2 has completed
epoch 3 has completed
epoch 4 has completed
epoch 5 has completed
epoch 6 has completed
epoch 7 has completed
epoch 8 has completed
epoch 9 has completed
epoch 10 has completed
epoch 11 has completed
epoch 12 has completed
epoch 13 has completed
epoch 14 has completed
epoch 15 has completed
epoch 16 has completed
epoch 17 has completed
epoch 18 has completed
epoch 19 has completed
epoch 20 has completed
epoch 21 has completed
epoch 22 has completed
epoch 23 has completed
epoch 24 has completed
epoch 25 has completed
0.8916


In [3]:
file_name = '../data/neural_network_lsh.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(neural_network_lsh, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "../data/neural_network_lsh.pkl"


In [2]:
function_num = 1
table_num = 1

top_k = 1-(1-(1/2**function_num))**table_num

epochs = 25
neural_network_wta = Network()
neural_network_wta.addLayer(LayerWTA(input_size, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_wta, epoch_time_wta] = neural_network_wta.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs, progress=True)
y_hat = neural_network_wta.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

Epoch  1: 100% |██████████| 60000/60000, Accuracy: 0.921;
Epoch  2: 100% |██████████| 60000/60000, Accuracy: 0.960;
Epoch  3: 100% |██████████| 60000/60000, Accuracy: 0.968;
Epoch  4: 100% |██████████| 60000/60000, Accuracy: 0.974;
Epoch  5: 100% |██████████| 60000/60000, Accuracy: 0.977;
Epoch  6: 100% |██████████| 60000/60000, Accuracy: 0.980;
Epoch  7: 100% |██████████| 60000/60000, Accuracy: 0.982;
Epoch  8: 100% |██████████| 60000/60000, Accuracy: 0.984;
Epoch  9: 100% |██████████| 60000/60000, Accuracy: 0.985;
Epoch 10: 100% |██████████| 60000/60000, Accuracy: 0.986;
Epoch 11: 100% |██████████| 60000/60000, Accuracy: 0.988;
Epoch 12: 100% |██████████| 60000/60000, Accuracy: 0.989;
Epoch 13: 100% |██████████| 60000/60000, Accuracy: 0.990;
Epoch 14: 100% |██████████| 60000/60000, Accuracy: 0.990;
Epoch 15: 100% |██████████| 60000/60000, Accuracy: 0.991;
Epoch 16: 100% |██████████| 60000/60000, Accuracy: 0.992;
Epoch 17: 100% |██████████| 60000/60000, Accuracy: 0.992;
Epoch 18: 100%

0.9815


In [ ]:
neural_network_lsh = Network()
neural_network_lsh.addLayer(LayerLSH(input_size, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_lsh, epoch_time_lsh] = neural_network_lsh.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs, progress=True)
y_hat = neural_network_lsh.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

In [ ]:
accuracy_standard = [
    0.83175, 0.84462, 0.844483, 0.85405723, 0.8647384, 
    0.8607931333, 0.86073, 0.870734, 0.8844062, 0.897336963, 
    0.90, 0.98863333, 0.98968333, 0.99053335, 0.9911, 
    0.9916833, 0.99235, 0.9928833, 0.99333334, 0.99378335, 
    0.99418336, 0.99465, 0.9950167, 0.9954, 0.99565]

In [ ]:
accuracy_dropout = [
    0.7931333, 0.95996666, 0.9684167, 0.97365, 0.97718334, 
    0.98, 0.9818, 0.98391664, 0.9853, 0.98645, 
    0.98753333, 0.98863333, 0.98968333, 0.99053335, 0.9911, 
    0.9916833, 0.99235, 0.9928833, 0.99333334, 0.99378335, 
    0.99418336, 0.99465, 0.9950167, 0.9954, 0.99565]


In [ ]:
accuracy_wta = [
    0.9209, 0.95996666, 0.9684167, 0.97365, 0.97718334, 
    0.98, 0.9818, 0.98391664, 0.9853, 0.98645, 
    0.98753333, 0.98863333, 0.98968333, 0.99053335, 0.9911, 
    0.9916833, 0.99235, 0.9928833, 0.99333334, 0.99378335, 
    0.99418336, 0.99465, 0.9950167, 0.9954, 0.99565]

In [3]:
accuracy_lsh = [
    0.91958336, 0.96806667, 0.9749667, 0.97738335, 0.9804,
    0.9818, 0.9853833, 0.98626665, 0.9885, 0.99045,
    0.9910667, 0.99095, 0.9919, 0.99288336, 0.9927,
    0.99295, 0.9940333, 0.9965333, 0.99828336, 0.99891666,
    0.99885, 0.9994, 0.99916664, 0.99965, 0.99885]

In [6]:
np.random.rand(25)

array([0.5362841 , 0.89039384, 0.79073547, 0.48260481, 0.17024111,
       0.06399728, 0.25115594, 0.17790812, 0.14406284, 0.57336963,
       0.17185621, 0.64811644, 0.6769215 , 0.24364536, 0.9993783 ,
       0.11331977, 0.08956745, 0.60139986, 0.55084815, 0.54334038,
       0.07638159, 0.45344607, 0.70810507, 0.79106681, 0.87139961])

In [9]:
1 - np.zeros((1,12), dtype=bool)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])